In [1]:
%cd ..

/home/jovyan/work/spectrum/c_final/v1


In [2]:
import pandas as pd
import base
import os

2024-02-27 09:42:27.607265: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-27 09:42:27.642146: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 09:42:27.642185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 09:42:27.643035: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 09:42:27.648728: I tensorflow/core/platform/cpu_feature_guar

In [2]:
SEED = 42
base.fix_seed(SEED)

desired_directory = '/home/jovyan/work/spectrum/'
os.chdir(desired_directory)

In [65]:
df = base.import_sqlite3_data('Data/01_sqlite_data/ms2_rikentest2.db')
df['dataset'] = df['dataset'].str.replace('data/', '', regex=False)

In [67]:
df['id'] = df['dataset'].apply(base.get_fileid).astype('float')
df = df[~df['Ontology'].str.contains('IS \(')]

df['Comment'] = df['Comment'].fillna('NC')
df = df[~(df['Comment'].str.contains('IS \(') | df['Comment'].str.contains('d') | df['Comment'].str.contains('two peaks'))]

/tmp/ipykernel_11811/2320284797.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Comment'] = df['Comment'].fillna('NC')


In [70]:
#データをneg / posに分割
df_pos = df[df['Adducttype'].str.contains('\]\+')  |  df['Adducttype'].str.contains('\]2\+')]
df_neg = df[df['Adducttype'].str.contains('\]\-')]
print(len(df), len(df_pos)+len(df_neg), len(df_neg), len(df_pos))

18834 18834 9501 9333


In [92]:
df_neg_exp_ = base.spectrum_tidy_rl(df_neg)
df_neg_neuloss_ = base.neutralloss_tidy_rl(df_neg)
df_neg_data = base.spectrum_neutralloss_table_rl(df_neg_exp_, df_neg_neuloss_)    
df_neg_data['Ontology'] = df_neg_data['Ontology'].str.replace(r'-', '_')

In [94]:
df_pos = df_pos[~df_pos['MSMSspectrum'].isna()]

In [95]:
df_pos_exp_ = base.spectrum_tidy_rl(df_pos)
df_pos_neuloss_ = base.neutralloss_tidy_rl(df_pos)
df_pos_data = base.spectrum_neutralloss_table_rl(df_pos_exp_, df_pos_neuloss_)    
df_pos_data['Ontology'] = df_pos_data['Ontology'].str.replace(r'-', '_')

In [96]:
df_pos_data.to_csv('data/05_rikenlipidomics/pos/df_rikenripidomics_pos.csv', index=False) 
df_neg_data.to_csv('data/05_rikenlipidomics/neg/df_rikenripidomics_neg.csv', index=False)